# *Algo 3.1 for the top down model*

In [1]:
import logging
import numpy as np
from scipy.special import hyp1f1, iv
from scipy.optimize import brentq, minimize_scalar
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
from typing import Optional, Tuple, Dict, List
from dataclasses import dataclass
import warnings
import time
import math

ModelParams = Dict[str, float]

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('default_simulation.log')
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class SimulationConfig:
    """Configuration for numerical methods and tolerances"""
    # Numerical methods and tolerances
    # Constants for numerical stability and convergence
    eps_zero: float = 1e-12
    eps_small: float = 1e-8
    # Maximum iterations for numerical methods
    max_iterations: int = 1000
    integration_points: int = 500
    root_finding_maxiter: int = 100
    # Number of grid points for numerical integration and interpolation
    grid_points: int = 500
    # Tolerance for numerical methods
    tolerance_scale: float = 1e-6

config = SimulationConfig()

In [2]:
def validate_parameters(model_params: ModelParams) -> bool:
    """Validate CIR parameters and check Feller condition"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"] 
        sigma = model_params["sigma"]
        
        assert kappa > 0, "kappa must be positive"
        assert theta > 0, "theta must be positive" 
        assert sigma > 0, "sigma must be positive"
        
        feller_condition = 2 * kappa * theta >= sigma**2
        if not feller_condition:
            logger.warning(f"Feller condition violated: 2kappa*theta={2*kappa*theta:.6f} < sigma^2={sigma**2:.6f}")
            
        logger.info(f"Parameters: κ={kappa:.6f}, θ={theta:.6f}, σ={sigma:.6f}")
        logger.info(f"Feller condition {'satisfied' if feller_condition else 'violated'}")
        
        return feller_condition
        
    except Exception as e:
        logger.error(f"Parameter validation failed: {e}")
        raise

In [3]:
def _hyp1f1(a: float, b: float, z: float, method: str = 'primary') -> float:
    """Hypergeometric function evaluation with fallbacks"""
    try:
        # Primary method
        if method == 'primary' and abs(z) < 100 and abs(a) < 50:
            result = hyp1f1(a, b, z)
            if np.isfinite(result) and abs(result) < 1e10:
                return result
        
        # # Fallback for large arguments using series truncation
        # if abs(z) > 100 or abs(a) > 50:
        #     logger.debug(f"Using series approximation for hyp1f1({a:.3f}, {b:.3f}, {z:.3f})")
        #     # For large z, use asymptotic expansion if a is negative integer-like
        #     if a < 0 and abs(a - round(a)) < config.eps_small:
        #         n = int(abs(round(a)))
        #         if n < 20:  # Finite series
        #             result = sum((a + k) * z**k / math.factorial(k) / (b + k) 
        #                        for k in range(n + 1))
        #             return result
        
        # Fallback resort: clip extreme values
        a_clip = np.clip(a, -50, 50)
        z_clip = np.clip(z, -100, 100)
        result = hyp1f1(a_clip, b, z_clip)
        
        if not np.isfinite(result):
            logger.warning(f"hyp1f1 returned non-finite: a={a}, b={b}, z={z}")
            return 1.0  
            
        return result
        
    except Exception as e:
        logger.warning(f"hyp1f1 evaluation failed: {e}, using fallback")
        return 1.0

In [4]:
def hyp1f1_derivative_a(a: float, b: float, z: float, delta: float = None) -> float:
    """Numerical derivative of hypergeometric function"""
    if delta is None:
        delta = max(config.eps_small, abs(a) * config.tolerance_scale)
    
    try:
        f_plus = _hyp1f1(a + delta, b, z)
        f_minus = _hyp1f1(a - delta, b, z)
        derivative = (f_plus - f_minus) / (2 * delta)
        
        if not np.isfinite(derivative):
            # Try smaller delta
            delta *= 0.1
            f_plus = _hyp1f1(a + delta, b, z)
            f_minus = _hyp1f1(a - delta, b, z)
            derivative = (f_plus - f_minus) / (2 * delta)
            
        return derivative if np.isfinite(derivative) else 0.0
        
    except Exception as e:
        logger.warning(f"Derivative computation failed: {e}")
        return 0.0

In [5]:
def laplace_transform_G(H: float, y: float, model_params: ModelParams) -> float:
    """Laplace transform computation"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        a = H / kappa
        b = 2 * kappa * theta / sigma**2
        z1 = 2 * kappa * y / sigma**2
        z2 = 2 * kappa * H / sigma**2
        
        logger.debug(f"Laplace transform: a={a:.6f}, b={b:.6f}, z1={z1:.6f}, z2={z2:.6f}")
        
        numerator = _hyp1f1(a, b, z1)
        denominator = _hyp1f1(a, b, z2)
        
        if abs(denominator) < config.eps_zero:
            logger.warning("Near-zero denominator in Laplace transform")
            return 0.99  # Safe fallback
            
        result = numerator / denominator
        
        # Stability check
        if not (0 <= result <= 1):
            logger.warning(f"Laplace transform out of bounds: {result:.6f}")
            result = np.clip(result, config.eps_zero, 1 - config.eps_zero)
            
        return result
        
    except Exception as e:
        logger.error(f"Laplace transform failed: {e}")
        return 0.5  # Conservative fallback

In [6]:
def compute_coefficients(y: float, H: float, model_params: ModelParams, n_roots: int = 15) -> Tuple[List[float], List[float]]:
    """Compute eigenfunction expansion coefficients as given by eqn 20 in section 4.2. root finding given in appendix B of the paper"""
    start_time = time.time()
    
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        c_bar = 2 * kappa * theta / sigma**2
        H_bar = 2 * kappa * H / sigma**2
        y_bar = 2 * kappa * y / sigma**2

        if c_bar < 1 or H_bar < c_bar:
            logger.error(f"Invalid parameters: c_bar={c_bar:.6f}, H_bar={H_bar:.6f}, y_bar={y_bar:.6f}")
            return [], []
        
        logger.debug(f"Computing coefficients: c_bar={c_bar:.6f}, H_bar={H_bar:.6f}, y_bar={y_bar:.6f}")
        
        def objective(alpha):
            return _hyp1f1(alpha, c_bar, H_bar)
        
        roots = []
        n = 0
        alpha_prev = 0

        # Find the first root in (-c_bar / H_bar, 0)
        left = -c_bar / H_bar
        right = 0
        try:
            alpha_curr = brentq(objective, left, right)
            roots.append(alpha_curr)
        except Exception as e:
            logger.error(f"Failed to find first root: {e}")
            return [], []

        # Find subsequent roots
        while len(roots) < n_roots:
            n += 1
            # Predict the next root using extrapolation
            alpha_hat = 2 * alpha_curr - alpha_prev

            # Inner loop to bracket the root
            bracket_found = False
            for attempt in range(20):  
                try:
                    test_val = objective(alpha_hat - 1)
                    if np.sign(test_val) == (-1)**n:
                        alpha_hat = alpha_hat - 1
                    else:
                        bracket_found = True
                        break
                except:
                    alpha_hat = alpha_hat - 1
                    if attempt > 10:  # Give up after many attempts
                        break
            
            if not bracket_found:
                logger.warning(f"Could not bracket root {n+1}, stopping at {len(roots)} roots")
                break
                
            try:
                alpha_next = brentq(objective, alpha_hat - 1, alpha_hat)
                roots.append(alpha_next)
                alpha_prev = alpha_curr
                alpha_curr = alpha_next
            except Exception as e:
                logger.warning(f"Failed to find root {n+1}: {e}, stopping at {len(roots)} roots")
                break
        
        if len(roots) < 5:
            logger.error(f"Insufficient roots found: {len(roots)}")
            return [], []
        
        # Compute coefficients
        eta_n, beta_n = [], []
        for alpha in roots:
            try:
                eta = -kappa * alpha
                eta_n.append(eta)
                
                numerator = _hyp1f1(alpha, c_bar, y_bar)
                derivative = hyp1f1_derivative_a(alpha, c_bar, H_bar)
                
                if abs(derivative) < config.eps_zero:
                    logger.warning(f"Near-zero derivative for alpha={alpha:.6f}")
                    beta = 0.0
                else:
                    beta = -numerator / (alpha * derivative)
                    
                beta_n.append(beta)
                
            except Exception as e:
                logger.warning(f"Coefficient computation failed for alpha={alpha:.6f}: {e}")
                eta_n.append(0.0)
                beta_n.append(0.0)
        
        # Filter out problematic coefficients
        valid_indices = [i for i, (eta, beta) in enumerate(zip(eta_n, beta_n)) 
                        if np.isfinite(eta) and np.isfinite(beta)]
        
        eta_n = [eta_n[i] for i in valid_indices]
        beta_n = [beta_n[i] for i in valid_indices]
        
        logger.debug(f"Computed {len(eta_n)} valid coefficients in {time.time() - start_time:.3f}s")
        
        return eta_n, beta_n
        
    except Exception as e:
        logger.error(f"Coefficient computation failed: {e}")
        return [], []

In [7]:
def compute_survival_probability_p1(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Compute survival probability"""
    try:
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients available, using fallback")
            return 0.5
        
        p1 = sum(beta * np.exp(-eta * tau) for eta, beta in zip(eta_n, beta_n) 
                if np.isfinite(beta) and np.isfinite(eta))
        
        # Stability bounds
        p1 = np.clip(p1, config.eps_zero, 1 - config.eps_zero)
        
        logger.debug(f"Survival probability: {p1:.6f}")
        return p1
        
    except Exception as e:
        logger.error(f"Survival probability computation failed: {e}")
        return 0.5

In [8]:
def cir_transition_density_g(x: float, y: float, t: float, model_params: ModelParams) -> float:
    """CIR transition density computation"""
    try:
        kappa = model_params["kappa"]
        theta = model_params["theta"]
        sigma = model_params["sigma"]
        
        if t <= 0 or x < 0 or y < 0:
            return 0.0
            
        q = 2 * kappa * theta / sigma**2 - 1
        exp_term = np.exp(-kappa * t)
        a = 2 * kappa / (sigma**2 * (1 - exp_term))
        b = a * exp_term
        
        by = b * y + config.eps_zero
        ax = a * x + config.eps_zero
        
        # Compute in log space for stability
        log_factor = np.log(a) - (ax + b * y)
        log_power = (q / 2) * (np.log(ax) - np.log(by))
        
        # Bessel function argument
        bessel_arg = 2 * np.sqrt(a * b * x * y)
        if bessel_arg > 100:  # Use asymptotic approximation
            log_bessel = bessel_arg - 0.5 * np.log(2 * np.pi * bessel_arg)
        else:
            bessel_val = iv(q, bessel_arg)
            if bessel_val <= 0:
                return 0.0
            log_bessel = np.log(bessel_val)
        
        log_density = log_factor + log_power + log_bessel
        
        if log_density > 50:  # Prevent overflow
            return 0.0
            
        return np.exp(log_density)
        
    except Exception as e:
        logger.debug(f"CIR density computation failed: {e}")
        return 0.0

In [9]:
def select_dominating_intensity_H(y: float, model_params: ModelParams, 
                              t_C: float = 1.0, t_eta: float = 1.8, buffer: float = 0.01) -> float:
    """Find optimal threshold H"""
    try:
        theta = model_params["theta"]
        
        H_max = max(y * 3, theta * 2, 1.0)
        H_min = max(y + config.eps_small, theta + buffer, config.eps_small)
        
        def objective(H):
            try:
                G = laplace_transform_G(H, y, model_params)
                
                numerator = H * (t_C + G * t_eta)
                denominator = 1 - G
                
                if denominator < config.eps_zero:
                    return 1e10
                    
                return numerator / denominator
                
            except:
                return 1e10
        
        result = minimize_scalar(objective, bounds=(H_min, H_max), method='bounded')
        
        if not result.success:
            logger.warning("H optimization failed, using heuristic")
            H_star = max(y * 1.5, theta * 1.2)
        else:
            H_star = result.x
        
        # Apply safety constraints
        H_final = max(H_star, y + config.eps_small, theta + buffer)
        
        logger.debug(f"Optimal H: {H_final:.6f} (y={y:.6f})")
        return H_final
        
    except Exception as e:
        logger.error(f"H optimization failed: {e}")
        return max(y * 1.5, theta * 1.2, 1.0)


In [10]:
def sample_conditional_intensity_from_f(y: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample from conditional transition density"""
    try:
        theta = model_params["theta"]
        nu_scale = max(y, theta, 0.1)
        x_max = max(nu_scale * 5, 10.0)  # Reasonable upper bound
        
        n_points = config.integration_points
        x_grid = np.linspace(config.eps_zero, x_max, n_points, endpoint=False)
        
        # Compute transition density
        g_vals = np.array([cir_transition_density_g(x, y, tau, model_params) for x in x_grid])
        
        # Compute convolution integral (simplified for robustness)
        eta_n, beta_n = compute_coefficients(y, H, model_params)
        eta_n = np.array(eta_n)
        beta_n = np.array(beta_n)

        s_grid = np.linspace(0, tau, config.grid_points, endpoint=False)

        def u(s):
            s = np.atleast_1d(s)
            return np.sum(beta_n[:, None] * eta_n[:, None] * np.exp(-eta_n[:, None] * s[None, :]), axis=0)

        
        conv_vals = np.empty_like(x_grid)

        for i, xi in enumerate(x_grid):
            g_shift = np.array([
                cir_transition_density_g(xi, H, tau - sj, model_params)
                for sj in s_grid
            ])
            conv_vals[i] = cumulative_trapezoid(g_shift * u(s_grid), s_grid)[-1]

        p1 = compute_survival_probability_p1(y, H, tau, model_params)
            
        f_vals = (g_vals - conv_vals) / p1
        f_vals = np.clip(f_vals, config.eps_zero, None)  # Avoid negative densities
        f_vals /= np.sum(f_vals)  # Normalize

        if np.sum(f_vals) <= config.eps_zero:
            logger.warning("Sampling failed, using fallback")
            return max(y * np.random.uniform(0.5, 1.5), config.eps_small)
        

        cdf_vals = cumulative_trapezoid(f_vals, x_grid, initial=0)
        # cdf_vals = np.cumsum((f_vals[:-1] + f_vals[1:]) * 0.5 * np.diff(x_grid))
        cdf_vals = np.concatenate(([0.0], cdf_vals))
        total_mass = cdf_vals[-1]
        if total_mass <= 0:
            return y
        cdf_vals /= total_mass

        if cdf_vals[-1] > config.eps_zero:
            cdf_vals /= cdf_vals[-1]
        else:
            # Uniform fallback
            cdf_vals = np.linspace(0, 1, len(cdf_vals))
        
        # Ensure monotonicity
        for i in range(1, len(cdf_vals)):
            cdf_vals[i] = max(cdf_vals[i], cdf_vals[i-1])
        
        # Sample
        u = np.random.uniform()
        try:
            inv_cdf = interp1d(cdf_vals, x_grid, bounds_error=False, 
                             fill_value=(x_grid[0], x_grid[-1]))
            sample = float(inv_cdf(u))
        except:
            # Linear interpolation fallback
            idx = np.searchsorted(cdf_vals, u)
            idx = min(idx, len(x_grid) - 1)
            sample = x_grid[idx]
        
        # Final bounds check
        sample = max(sample, config.eps_small)
        
        logger.debug(f"Sampled intensity: {sample:.6f}")
        return sample
        
    except Exception as e:
        logger.error(f"Conditional sampling failed: {e}")
        return max(y * np.random.uniform(0.8, 1.2), config.eps_small)


In [11]:
def sample_hitting_time_from_v(nu_t: float, H: float, tau: float, model_params: ModelParams) -> float:
    """Sample hitting time"""
    try:
        eta_n, beta_n = compute_coefficients(nu_t, H, model_params)
        
        if not eta_n:
            logger.warning("No coefficients for hitting time, using uniform")
            return np.random.uniform(0, tau)
        
        P = 1 - compute_survival_probability_p1(nu_t, H, tau, model_params)
        P = max(P, config.eps_zero)
        
        U = np.random.uniform(0, P)
        
        def objective(s):
            integral = sum(beta * (1 - np.exp(-eta * s)) for beta, eta in zip(beta_n, eta_n)
                          if eta > config.eps_zero)
            return integral - U

        try:
            result = brentq(objective, 0, tau)
            return min(result, tau)
        except:
            logger.warning("Hitting time root finding failed, using approximation")
            return np.random.uniform(0, tau * 0.8)
            
    except Exception as e:
        logger.error(f"Hitting time sampling failed: {e}")
        return np.random.uniform(0, tau * 0.5)

In [12]:
def sample_loss() -> float:
    """Sample loss from a uniform distribution as per the original model"""
    return np.random.uniform(0.24, 0.96)

In [13]:
def simulate_next_default_step(
    t_prev: float,
    lambda_prev: float,
    T_max: float,
    model_params: ModelParams
) -> Optional[Tuple[float, float]]:
    """
    Simulates a single default step in the top-down model.

    Parameters:
        t_prev: float
            Previous default time T_{n-1}
        lambda_prev: float
            Intensity after last jump, λ_{T_{n-1}} 
        T_max: float
            Final time horizon for simulation
        model_params: dict
            Contains CIR and model parameters: kappa, theta, sigma, gamma

    Returns:
        Optional[Tuple[float, float]]:
            If a default occurs before T_max:
                (T_n, ν_{T_n})
            Else:
                None (no more defaults)
    """
    try:
        # STEP 1: Initialize y := λ_{T_{n-1}}, t := T_{n-1}
        y = max(lambda_prev, config.eps_small) # Ensure y is positive
        t = t_prev 
        
        max_inner_iterations = 50  
        
        for iteration in range(max_inner_iterations):
            # STEP 2: Select H>y
            H = select_dominating_intensity_H(y, model_params)
            logger.info(f"Iteration {iteration + 1}: Selected H={H:.6f} for y={y:.6f}")
            
            # STEP 3: Sample τ ∼ Exp(H), propose next interarrival time
            tau = np.random.exponential(scale = 1/H)
            logger.debug(f"Sampled interarrival time τ={tau:.6f} from Exp(H) with H={H:.6f}")
            
            # STEP 4: Compute survival probability p1 = P(ν stays below H up to τ)
            p1 = compute_survival_probability_p1(y, H, tau, model_params)
            logger.info(f"Computed survival probability p1={p1:.6f} for y={y:.6f}, H={H:.6f}, τ={tau:.6f}")
            
            # STEP 5: First rejection test with u1 ∼ Unif(0,1)
            u1 = np.random.uniform()
            
            if u1 > p1:
                # Process hits H → sample σ_H from hitting time density
                logger.info(f"Process hits boundary H={H:.6f} at t={t:.6f}, sampling hitting time")
                sigma_H = sample_hitting_time_from_v(y, H, tau, model_params)
                t += sigma_H
                
                # STEP 8: Check if t exceeds T_max
                if t > T_max:
                    return None
                    
                y = H  # Update intensity and repeat
                continue
            else:
                # Process survives to τ
                t += tau
                logger.info(f"Process survives to t={t:.6f}, sampling conditional intensity")
                
                # STEP 8: Check if t exceeds T_max
                if t > T_max:
                    return None
                
                # STEP 6: Sample ν_{T_n} from conditional density f(.; ν_t, H, τ)
                nu_Tn = sample_conditional_intensity_from_f(y, H, tau, model_params)
                logger.debug(f"Sampled conditional intensity ν_{t}={nu_Tn:.6f} at t={t:.6f}")
                
                # STEP 7: Second rejection test using u2 ∼ Unif(0,1)
                u2 = np.random.uniform()
                
                if u2 <= nu_Tn / H:
                    # Accept default at T_n = t
                    # ell_n = sample_loss()  # Draw loss ℓ_n ∼ loss_distribution()
                    
                    # logger.debug(f"Default at t={t:.6f}, ν={nu_Tn:.6f}, loss={ell_n:.6f}")
                    logger.info(f"Default accepted at t={t:.6f}, ν={nu_Tn:.6f}.")
                    return (t, nu_Tn)
                else:
                    # Reject - update ν_t and go back to step 2
                    y = nu_Tn
                    continue
        
        logger.warning(f"Maximum inner iterations reached at t={t:.6f}")
        return None
        
    except Exception as e:
        logger.error(f"Error in simulate_next_default_step: {e}")
        return None

def simulate_defaults(
    lambda_0: float,
    T_max: float,
    model_params: ModelParams,
    max_defaults: int = 1000
) -> List[Tuple[float, float]]:
    """
    Main simulation function that generates a sequence of default times and losses.
    
    Parameters:
        lambda_0: float
            Initial intensity λ_0
        T_max: float
            Time horizon
        model_params: ModelParams
            Dictionary containing kappa, theta, sigma, gamma
        max_defaults: int
            Maximum number of defaults to simulate
            
    Returns:
        List[Tuple[float, float]]: List of (default_time, loss) pairs
    """
    logger.info(f"Starting default simulation: λ_0={lambda_0:.6f}, T_max={T_max:.2f}")
    
    # Validate parameters
    if not validate_parameters(model_params):
        logger.warning("Proceeding despite Feller condition violation")
    
    defaults = []
    t = 0.0
    lam = max(lambda_0, config.eps_small)
    gamma = model_params.get("gamma", 2.99)  # Default jump size
    start_time = time.time()
    
    try:
        while t < T_max and len(defaults) < max_defaults:
            result = simulate_next_default_step(t, lam, T_max, model_params)
            
            if result is None:
                break
                
            Tn, nu_Tn = result
            ell_n = sample_loss()
            lam = nu_Tn + gamma * ell_n  # Update intensity at default with jump
            t = Tn
            defaults.append((t, lam, ell_n))
            
            if len(defaults) % 10 == 0:
                logger.info(f"Generated {len(defaults)} defaults, t={t:.4f}, λ={lam:.6f}")
        
        elapsed_time = time.time() - start_time
        logger.info(f"Simulation completed: {len(defaults)} defaults in {elapsed_time:.2f}s")
        
        return defaults
        
    except KeyboardInterrupt:
        logger.info("Simulation interrupted by user")
        return defaults
    except Exception as e:
        logger.error(f"Critical simulation error: {e}")
        return defaults

In [14]:
# paper parameters for testing
model_params = {
    "kappa": 2.62,    # Mean reversion speed
    "theta": 1.61,    # Long-term mean
    "sigma": 0.62,    # Volatility
    "gamma": 0.99     # Jump size multiplier
}

defaults = simulate_defaults(
    lambda_0=0.7,
    T_max=5,
    model_params=model_params,
    max_defaults=100
)

print(f"Generated {len(defaults)} defaults:")
for i, (t, lam, loss) in enumerate(defaults): 
    print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")


2025-05-31 14:59:47,243 - INFO - Starting default simulation: λ_0=0.700000, T_max=5.00
2025-05-31 14:59:47,243 - INFO - Parameters: κ=2.620000, θ=1.610000, σ=0.620000
2025-05-31 14:59:47,244 - INFO - Feller condition satisfied
2025-05-31 14:59:47,244 - INFO - Iteration 1: Selected H=2.295761 for y=0.700000
2025-05-31 14:59:47,245 - INFO - Computed survival probability p1=0.999443 for y=0.700000, H=2.295761, τ=0.051421
2025-05-31 14:59:47,245 - INFO - Process survives to t=0.051421, sampling conditional intensity
2025-05-31 14:59:48,114 - INFO - Default accepted at t=0.051421, ν=0.720000.
2025-05-31 14:59:48,115 - INFO - Iteration 1: Selected H=2.471099 for y=1.361046
2025-05-31 14:59:48,115 - INFO - Computed survival probability p1=0.981699 for y=1.361046, H=2.471099, τ=0.439324
2025-05-31 14:59:48,116 - INFO - Process survives to t=0.490745, sampling conditional intensity
2025-05-31 14:59:49,006 - INFO - Default accepted at t=0.490745, ν=1.500000.
2025-05-31 14:59:49,006 - INFO - Iter

Generated 100 defaults:
  1: t=0.051421, itensity=1.361046, loss=0.647521
  2: t=0.490745, itensity=1.962508, loss=0.467180
  3: t=0.862888, itensity=2.465714, loss=0.531024
  4: t=0.937306, itensity=2.819066, loss=0.456546
  5: t=1.000672, itensity=3.118269, loss=0.444602
  6: t=1.297904, itensity=3.832421, loss=0.941850
  7: t=1.633900, itensity=4.359097, loss=0.880398
  8: t=1.653682, itensity=4.879193, loss=0.525349
  9: t=1.686819, itensity=5.529953, loss=0.903757
  10: t=2.197755, itensity=390639559844993630208.000000, loss=0.912564
  11: t=2.197755, itensity=918002965635735093248.000000, loss=0.468242
  12: t=2.197755, itensity=2307688781765914367361024.000000, loss=0.382754
  13: t=2.197755, itensity=83492177205750022663045120.000000, loss=0.400341
  14: t=2.197755, itensity=346492535403862619821441024.000000, loss=0.829606
  15: t=2.197755, itensity=464299997441175906437562368.000000, loss=0.428776
  16: t=2.197755, itensity=187665402845737972981139767296.000000, loss=0.512889

In [15]:
L = sum(loss for _, _, loss in defaults)
print(f"Total loss: {L:.6f}")

Total loss: 58.963041


In [16]:
model_params = {
    "kappa": 2.62,    # Mean reversion speed - from paper
    "theta": 1.61,    # Long-term mean - from paper
    "sigma": 0.62,    # Volatility - from paper
    "gamma": 0.99     # Jump size multiplier - paper's value is 2.99 but we use 0.99 for stability 
    # !TODO: Check why Laplace transform blows up with gamma >~ 1.5. 
    # All further calculations of survival probabilities and sampling from f,v are unstable.
}

L_5 = []

for _ in range(10):
    defaults = simulate_defaults(
        lambda_0=0.7,
        T_max=5,
        model_params=model_params,
        max_defaults=100
    )

    print(f"Generated {len(defaults)} defaults:")
    for i, (t, lam, loss) in enumerate(defaults): 
        print(f"  {i+1}: t={t:.6f}, itensity={lam:.6f}, loss={loss:.6f}")

    l_5 = sum(loss for _, _, loss in defaults)
    print(f"Total loss: {l_5:.6f}")

    L_5.append(l_5)

print(f"Average total loss over 10 runs: {np.mean(L_5):.6f} ± {np.std(L_5):.6f}")



2025-05-31 15:02:34,013 - INFO - Starting default simulation: λ_0=0.700000, T_max=5.00
2025-05-31 15:02:34,013 - INFO - Parameters: κ=2.620000, θ=1.610000, σ=0.620000
2025-05-31 15:02:34,014 - INFO - Feller condition satisfied
2025-05-31 15:02:34,015 - INFO - Iteration 1: Selected H=2.295761 for y=0.700000
2025-05-31 15:02:34,015 - INFO - Computed survival probability p1=0.999993 for y=0.700000, H=2.295761, τ=0.203033
2025-05-31 15:02:34,016 - INFO - Process survives to t=0.203033, sampling conditional intensity
2025-05-31 15:02:34,875 - INFO - Default accepted at t=0.203033, ν=2.460000.
2025-05-31 15:02:34,876 - INFO - Iteration 1: Selected H=3.742153 for y=3.168821
2025-05-31 15:02:34,876 - INFO - Computed survival probability p1=0.981436 for y=3.168821, H=3.742153, τ=1.029111
2025-05-31 15:02:34,877 - INFO - Process survives to t=1.232144, sampling conditional intensity
2025-05-31 15:02:35,818 - INFO - Iteration 2: Selected H=2.669356 for y=1.806228
2025-05-31 15:02:35,819 - INFO - 

Generated 7 defaults:
  1: t=0.203033, itensity=3.168821, loss=0.715981
  2: t=1.431052, itensity=2.714422, loss=0.842850
  3: t=2.391210, itensity=1.816965, loss=0.259561
  4: t=2.522616, itensity=2.394247, loss=0.600249
  5: t=2.851225, itensity=2.236071, loss=0.710862
  6: t=3.257768, itensity=2.673375, loss=0.419134
  7: t=4.854400, itensity=2.218753, loss=0.665407
Total loss: 4.214044


2025-05-31 15:02:46,434 - INFO - Default accepted at t=0.556577, ν=1.640000.
2025-05-31 15:02:46,435 - INFO - Iteration 1: Selected H=2.759874 for y=1.961505
2025-05-31 15:02:46,436 - INFO - Computed survival probability p1=0.998775 for y=1.961505, H=2.759874, τ=0.100374
2025-05-31 15:02:46,436 - INFO - Process survives to t=0.656951, sampling conditional intensity
2025-05-31 15:02:47,295 - INFO - Default accepted at t=0.656951, ν=1.440000.
2025-05-31 15:02:47,296 - INFO - Iteration 1: Selected H=2.995357 for y=2.300938
2025-05-31 15:02:47,297 - INFO - Computed survival probability p1=0.992471 for y=2.300938, H=2.995357, τ=0.139467
2025-05-31 15:02:47,297 - INFO - Process survives to t=0.796418, sampling conditional intensity
2025-05-31 15:02:48,157 - INFO - Iteration 2: Selected H=2.593050 for y=1.656675
2025-05-31 15:02:48,158 - INFO - Computed survival probability p1=0.996407 for y=1.656675, H=2.593050, τ=0.192186
2025-05-31 15:02:48,158 - INFO - Process survives to t=0.988604, samp

Generated 13 defaults:
  1: t=0.556577, itensity=1.961505, loss=0.324752
  2: t=0.656951, itensity=2.300938, loss=0.869634
  3: t=0.988604, itensity=1.729125, loss=0.271844
  4: t=1.529408, itensity=2.356438, loss=0.400442
  5: t=1.702835, itensity=2.328213, loss=0.495143
  6: t=1.883761, itensity=2.878765, loss=0.767769
  7: t=2.063426, itensity=3.274419, loss=0.864906
  8: t=2.407103, itensity=3.811167, loss=0.641394
  9: t=2.655825, itensity=3.504065, loss=0.830067
  10: t=2.968188, itensity=3.761512, loss=0.817866
  11: t=3.273179, itensity=3.777165, loss=0.775712
  12: t=3.394638, itensity=3.997353, loss=0.565791
  13: t=3.444715, itensity=4.012634, loss=0.257699
Total loss: 7.883020


2025-05-31 15:03:04,941 - INFO - Default accepted at t=1.842648, ν=1.540000.
2025-05-31 15:03:04,942 - INFO - Iteration 1: Selected H=2.706610 for y=1.872555
2025-05-31 15:03:04,942 - INFO - Computed survival probability p1=0.996172 for y=1.872555, H=2.706610, τ=0.155367
2025-05-31 15:03:04,943 - INFO - Process survives to t=1.998014, sampling conditional intensity
2025-05-31 15:03:05,817 - INFO - Iteration 2: Selected H=2.594630 for y=1.660000
2025-05-31 15:03:05,818 - INFO - Computed survival probability p1=0.990679 for y=1.660000, H=2.594630, τ=0.268592
2025-05-31 15:03:05,818 - INFO - Process survives to t=2.266606, sampling conditional intensity
2025-05-31 15:03:06,683 - INFO - Default accepted at t=2.266606, ν=2.100000.
2025-05-31 15:03:06,684 - INFO - Iteration 1: Selected H=3.185847 for y=2.539892
2025-05-31 15:03:06,684 - INFO - Computed survival probability p1=0.972266 for y=2.539892, H=3.185847, τ=0.570449
2025-05-31 15:03:06,685 - INFO - Process survives to t=2.837056, samp

...

Generated 7 defaults:
  1: t=1.024132, itensity=2.201846, loss=0.527118
  2: t=1.311361, itensity=3.007277, loss=0.924770
  3: t=2.143061, itensity=2.634261, loss=0.600264
  4: t=2.445221, itensity=2.712467, loss=0.744214
  5: t=3.142381, itensity=2.050437, loss=0.489698
  6: t=3.306835, itensity=2.522459, loss=0.456078
  7: t=4.137576, itensity=3.048738, loss=0.608033
Total loss: 4.350175


2025-05-31 15:04:25,305 - INFO - Default accepted at t=1.487526, ν=2.295761.
2025-05-31 15:04:25,306 - INFO - Iteration 1: Selected H=3.528188 for y=2.934218
2025-05-31 15:04:25,306 - INFO - Computed survival probability p1=0.977874 for y=2.934218, H=3.528188, τ=1.165941
2025-05-31 15:04:25,307 - INFO - Process hits boundary H=3.528188 at t=1.487526, sampling hitting time
2025-05-31 15:04:25,308 - INFO - Iteration 2: Selected H=4.078773 for y=3.528188
2025-05-31 15:04:25,309 - INFO - Computed survival probability p1=0.996895 for y=3.528188, H=4.078773, τ=0.052516
2025-05-31 15:04:25,309 - INFO - Process survives to t=1.897713, sampling conditional intensity
2025-05-31 15:04:26,171 - INFO - Default accepted at t=1.897713, ν=3.563470.
2025-05-31 15:04:26,172 - INFO - Iteration 1: Selected H=5.019363 for y=4.502136
2025-05-31 15:04:26,172 - INFO - Computed survival probability p1=0.989994 for y=4.502136, H=5.019363, τ=0.373707
2025-05-31 15:04:26,172 - INFO - Process survives to t=2.27142

Generated 11 defaults:
  1: t=1.487526, itensity=2.934218, loss=0.644906
  2: t=1.897713, itensity=4.502136, loss=0.948147
  3: t=2.476365, itensity=3.288674, loss=0.504647
  4: t=3.013406, itensity=2.818461, loss=0.251868
  5: t=3.463292, itensity=2.670877, loss=0.591127
  6: t=4.085549, itensity=1.907998, loss=0.254600
  7: t=4.173949, itensity=2.748675, loss=0.837045
  8: t=4.365763, itensity=2.890710, loss=0.365585
  9: t=4.546525, itensity=3.777568, loss=0.925015
  10: t=4.768771, itensity=3.210076, loss=0.342550
  11: t=4.913531, itensity=3.697292, loss=0.362437
Total loss: 6.027927


2025-05-31 15:04:36,795 - INFO - Iteration 2: Selected H=2.362843 for y=1.000000
2025-05-31 15:04:36,796 - INFO - Computed survival probability p1=0.999608 for y=1.000000, H=2.362843, τ=0.038601
2025-05-31 15:04:36,796 - INFO - Process survives to t=0.456942, sampling conditional intensity
2025-05-31 15:04:37,654 - INFO - Iteration 3: Selected H=2.418226 for y=1.200000
2025-05-31 15:04:37,655 - INFO - Computed survival probability p1=1.000000 for y=1.200000, H=2.418226, τ=0.043500
2025-05-31 15:04:37,655 - INFO - Process survives to t=0.500442, sampling conditional intensity
2025-05-31 15:04:38,526 - INFO - Iteration 4: Selected H=2.424353 for y=1.220000
2025-05-31 15:04:38,527 - INFO - Computed survival probability p1=1.000000 for y=1.220000, H=2.424353, τ=0.014726
2025-05-31 15:04:38,527 - INFO - Process survives to t=0.515168, sampling conditional intensity
2025-05-31 15:04:39,401 - INFO - Iteration 5: Selected H=2.443454 for y=1.280000
2025-05-31 15:04:39,402 - INFO - Computed surv

Generated 8 defaults:
  1: t=0.836105, itensity=2.565664, loss=0.834004
  2: t=0.921041, itensity=3.189641, loss=0.448870
  3: t=1.094715, itensity=2.687519, loss=0.588239
  4: t=2.433936, itensity=1.669082, loss=0.574831
  5: t=4.164980, itensity=1.484093, loss=0.327367
  6: t=4.557064, itensity=1.878434, loss=0.301449
  7: t=4.581844, itensity=2.418962, loss=0.322184
  8: t=4.955857, itensity=2.540646, loss=0.394619
Total loss: 3.791561
Average total loss over 10 runs: 6.031305 ± 2.775637
